In [2]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import datetime

# データ生成用モジュール
# クラスと関数だけ集めておくことにする


In [3]:

# ユーティリティ
# 全角→半角
def convert_zenkaku_to_hankaku(text):
    return text.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))

# 日付str → datetime
def get_formatted_date(datestr):
    return datetime.datetime.strptime(datestr,"%Y/%m/%d")


In [6]:
# データ整形クラス

# 演歌グループ

class EnkaScraper:
    
    def __init__(self, datestr, soup_url):
        self.df = pd.DataFrame()
        self.soup =  BeautifulSoup(open(soup_url),"html.parser")
        self.datestr = datestr
        
#         アーティスト名(演歌)
    def add_artist(self):
        self.df["artistname"]= "演歌"

    # 公演名: name
    def add_name(self):
        names = list(map(lambda x: convert_zenkaku_to_hankaku(x.string.strip().replace("\n","").replace("\u3000","")), self.soup.select(".Product__titleLink") ))
        self.df["name"] = names
        
        
    #  　フォーマットユーティリティ 2019-05-17 → 2019/05/17
    def convert_hyphen_to_slash(self):
        return self.datestr.replace("-", "/")
    
    # 先頭の./を取り除く ./2019-05-23 → 2019-05-23
    def format_datename(name):
        return name.replace("./", "")

        
    #公演日時
    # Timestamp YYYY/mm/dd
    def add_date(self):
        date_pattern = re.compile(r"(\d{1,4}\/\d{1,2}\/\d{1,2}|\d月\d+日|\d{1,3}\/\d+)")
        left_days = self.soup.select(".Product__time") #残り時間
        collected_at = self.convert_hyphen_to_slash() #収集日
        collected_at = get_formatted_date(collected_at)

        date_list = []
        for i , data in enumerate(self.df["name"]):
        #     商品名に日付表現があれば
            if(date_pattern.search(data)):
        #         見つかったやつから生成した日付がdate
                found = date_pattern.search(data)[0]
                found = found.replace("月", "/").replace("日", "")
                nums = [int(s) for s in found.split("/")]
                if(len(nums) <= 2): #年がなかったら
                    this_year = datetime.datetime.today().year
                    nums.insert(0,this_year)
                d = datetime.date(*nums)
                date_list.append(d)
        #       商品名に日付表現がなければ
            else:
        #         同じアイテムの残り日数/時間を収集日に加算したものがdate
                nokori =left_days[i].text.strip()
        #         日を含んでいれば収集日にその日づけを加算
                if "日" in nokori:
                    d = collected_at + datetime.timedelta(days=int(re.sub("\\D", "", nokori)))
                    date_list.append(d)
        #             含んでいなけれその日(時間は更新する必要ないので)
                else:
                    d = collected_at 
                    date_list.append(d)

        self.df["date"] = date_list
        # datetime.date型で収めてしまったのでtimestampに変換
        self.df["date"] = pd.to_datetime(self.df["date"])

    


    
    #  　フォーマットユーティリティ 2019-05-17 → 2019/05/17
    def convert_hyphen_to_slash(self):
        return self.datestr.replace("-", "/")
    
    #チケットデータ収集日  timestamp YYYY/mm/dd
    def add_collected_at(self):
        slash_date = self.convert_hyphen_to_slash()
        self.df["collected_at"] = datetime.datetime.strptime(slash_date,"%Y/%m/%d")
   
    #公演までの残り日数 int
    def add_left_days(self):
        self.df["left_days"] = (self.df["date"] -self.df["collected_at"]).dt.days  #日時差をint変換
    
    #  ジャニーズダミー 0 1
    def add_is_jannies(self):
        IS_JANNIES = 0
        self.df["is_jannies"] =IS_JANNIES
        
    #         入手可能ダミー
    def add_available(self):
        self.df["available"] =1

        
#         取引あたり価格
    # 価格について:price_per_unit/price_per_deal
    # 即決価格はないときもあればあるときもある
    # 現在価格は必ずある
    # 即決価格があれば即決価格を、なければ現在価格をprice_per_dealとして採用することにする
    # 現在価格のセレクタは.Product__priceValue.u-textRed 即決価格は.Product__priceValue
    def add_price_per_deal(self):
        price_list = []
        price_soup =self.soup.select(".Product__priceInfo")
        for s in price_soup:
            if(s.select(".Product__priceValue:not(.u-textRed)")):
                s = s.select(".Product__priceValue:not(.u-textRed)")[0].text.replace(" ", "").replace("円", "").replace(",", "")
                s = int(s)
                price_list.append(s)
            else:
                s = s.select(".Product__priceValue.u-textRed")[0].text.replace(" ", "").replace("円", "").replace(",", "")
                s =int(s)
                price_list.append(s)
        self.df["price_per_deal"] = price_list
    
#     連番
    def add_unit(self):
        unit_list = []
        r_pattern = re.compile(r"\d+枚")
        for i in self.df["name"]:
            if(r_pattern.search(i)):
                found = r_pattern.search(i)[0]
                num = int(re.sub("\\D", "", found))
                unit_list.append(num)
            else:
                unit_list.append(1)
        # r_pattern.search(text)
        # print(text)
        self.df["unit"] = unit_list
    
    #   単位あたり価格 intのつもり
    def add_price_per_unit(self):
        self.df["price_per_unit"] = self.df["price_per_deal"]/ self.df["unit"]

#     取引あたり価格/連番/単位あたり価格の順で ただのファサード
    def add_price_info_bundle(self):
        self.add_price_per_deal()
        self.add_unit() 
        self.add_price_per_unit()
        

    #　チケット転売施行ダミー 01
    # 収集日によって決定　施行後なら1、施行前なら0
    def add_after(self):
        ENFORCE_DATE = "2019-06-14"
        self.df["after"]=   (self.df["collected_at"] >=ENFORCE_DATE)*1
        
#  施行前後ダミー×アーティストグループダミーの交差項
    
    def add_cross_term(self):
        self.df["after_and_is_jannies"] = self.df["after"] * self.df["is_jannies"]

    
    #   (メインロジック):列の追加から書き出し
    def store_data(self):
        self.add_name() #名前
        self.add_date() #日付
        self.add_collected_at() #収集日
        self.add_left_days() #残り日数
        self.add_is_jannies() #グループダミー
        self.add_available() #購入可能
        self.add_price_info_bundle() #取引価格/枚数/単位価格
        self.add_after() #法施行前後ダミー
        self.add_cross_term() #交差項
        self.add_artist() #アーティストグループ
        self.drop_outlier()# 外れ値を落とす
        
    def drop_outlier(self):
        self.df = self.df[self.df["price_per_unit"] >= 1000]
        
        #     CSV書き出し
    def write_in_csv(self, filename):
        self.df.to_csv(filename,index=0)
    


In [7]:
# テスト

soup_url = "../細川たかし/2019-09-20/ticket-data1.html"
sc1 = EnkaScraper("2019-05-16",soup_url)
sc1.store_data()
sc1.df


,name,date,collected_at,left_days,is_jannies,available,price_per_deal,unit,price_per_unit,after,after_and_is_jannies,artistname
